# Check EBL data in Gammapy and in GRB files
Lara provides absorbed flux specturm - we want to check if this is compatible with the gammapy data

In [ ]:
import os
os.environ['GAMMAPY_EXTRA'] =r'../../input/gammapy-extra-master'
os.environ['GAMMAPY_DATA'] =r'../../input/gammapy-extra-master/datasets'
#print(os.getenv('GAMMAPY_EXTRA'))
#print(os.listdir(os.getenv('GAMMAPY_EXTRA')))

import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
from   astropy.table         import Table
from   astropy.io            import fits

from astropy.visualization import quantity_support

from gammapy.modeling.models import EBLAbsorptionNormSpectralModel
from gammapy.modeling.models import TemplateSpectralModel

## Read a GRB

In [ ]:
event_id = 20 # z = 
event_id = 343 # z = 0.13
# event_id = 7 # z= 0.42
# event_id = 676 # z= 2.17
data_dir = "../../input/lightcurves/LONG_FITS/"
filename = data_dir + "Event" + str(event_id)+".fits"
print("Reading data from ",filename)
hdul = fits.open(filename)
Eval     = Table.read(hdul,hdu=2)["Energies (afterglow)"].quantity
Eval     = np.asarray(Eval)*Eval.unit
tval     = Table.read(hdul,hdu=3)["Times (afterglow)"].quantity
tval     = np.asarray(tval)*tval.unit
hdr      = hdul[0].header
redshift = hdr['Z']
print(" Got a GRB at redshift ",redshift)

## Get original and absorbed flux from Lara

In [ ]:
# Convert flux table into a numpy array
def get_lara_flux(flux_table,flux_unit = u.Unit("1/(cm2 GeV s)")):
    icol_t = len(flux_table.colnames)          # column number - time
    jrow_E = len(flux_table[flux_table.colnames[0]]) # row number   
    flux   = np.zeros( (icol_t,jrow_E) )*flux_unit
    
    for i in range(0,icol_t):
        for j in range(0,jrow_E):
            flux[i][j] = flux_table[j][i]*flux_unit # transp!
            
    return flux

f_lara     = get_lara_flux(Table.read(hdul,hdu=4)) # Original flux
f_lara_abs = get_lara_flux(Table.read(hdul,hdu=5)) # Absorbed flux

# Read local EBL models

In [ ]:
from ebl import EBL_from_file
local_gilmore = EBL_from_file("data/"+"ebl_gilmore12.dat",debug=True)
local_dominguez = EBL_from_file("data/EBL_abs_RBelmont-dominguez-20170425.dat",debug=True)

## Find back EBL model - compare to local Gilmore model

In [ ]:
itime = 10 # Choose a time slice
def plot_ratio(E, flux, flux_abs, ax=None,
               max_reduction=1e-3,color="tab:blue",
               label="default",**kwargs):
               
    if (ax == None): fig, ax = plt.subplots()
               
    # remove zeroes from array
    id0  = np.where(flux>0)
    f    = flux[id0]
    fabs = flux_abs[id0]
    E    = E[id0]

    ratio = fabs/f
    for i,r in enumerate(ratio):
        if r>1: print("ratio>1 in bin ",i,":",ratio[ratio>1])
    
    idr = np.where(ratio>max_reduction)
    with quantity_support():
        ax.plot(E[idr],ratio[idr],marker="D",color=color,label=label,**kwargs)
    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.legend()
        
    return ax, ratio 

In [ ]:
itime=10
ax, ratio_lara = plot_ratio(Eval,f_lara[itime],f_lara_abs[itime],ls=":")

# The second plot should beperfectly superimposed onthe first one
from ebl import EBL_plot
EBL_plot(local_gilmore,zlist=redshift,ax=ax)

## Check Gammapy absorption models

In [ ]:
dominguez = EBLAbsorptionNormSpectralModel.read_builtin("dominguez", redshift=redshift)
franceschini = EBLAbsorptionNormSpectralModel.read_builtin("franceschini", redshift=redshift)
finke = EBLAbsorptionNormSpectralModel.read_builtin("finke", redshift=redshift)

### Compare Gammapy versus Renaud's data

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
energy_range = [10, 10000] * u.GeV
opts = dict(energy_range=energy_range, energy_unit="GeV", flux_unit="", ax=ax)
dominguez.plot(label="Dominguez 2011", **opts)
EBL_plot(local_dominguez,zlist=redshift,tags="Dominguez from file",
         nebin=20,ax=ax,marker="o",ls="",color="black",alpha=0.5)

In [ ]:
### Compare all absorptions, including local Gilmore

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))

energy_range = [10, 10000] * u.GeV
opts = dict(energy_range=energy_range, energy_unit="GeV", flux_unit="", ax=ax)

franceschini.plot(label="Franceschini 2008", **opts)
finke.plot(label="Finke 2010", **opts)
dominguez.plot(label="Dominguez 2011", **opts)

EBL_plot(local_gilmore,zlist=redshift,tags="Gilmore from file",ax=ax,marker="",color="red")


## Plot ratios of ratios (absorption)

In [ ]:
def plot_absorption_ratio(spectrum, flux2, Emin=0*u.GeV,Emax = 100*u.GeV,nplots=None,nmaxcol=4):
    
    idE = np.where( [(E>=Emin and E<=Emax) for E in Eval])
    
    import itertools
    ncols  = min(nmaxcol,nplots) # If nplots < nmaxcol, take nplots
    nrows  = int(nplots/ncols)+ 1*(nplots%ncols != 0)
    fig,ax= plt.subplots(nrows=nrows,ncols=ncols,sharex=True,figsize=(15,2*nrows))

    # Slice to plot
    nslice = len(flux2[0])
    islice = range(1,nslice-1,int(nslice/nplots)) # avoid slice 0

    with quantity_support():
        iplot=0
        for jrow, icol in itertools.product(range(nrows), range(ncols)):
            ax0 = ax[jrow][icol] if (nrows>1) else ax[icol]
            color = cm(1.*iplot/nslice)
            
            # If nothing to plot, blank space instead of empty plot
            if (iplot >= nplots):
                ax0.axis('off')
                continue
            else:
                idx = islice[iplot]
                t = tval[idx]
            label=str(round(t.value,1))+str(t.unit)
            
            ax0.plot(Eval[idE],spectrum[idx](Eval[idE])/flux2[idx][idE],color=color,
                           label=label,marker="o",ls=":")
            
            ax0.set_xscale("log")
            ax0.set_xlim(xmin=Emin,xmax=Emax)
            ax0.legend()
            ax0.tick_params(which='major', length=10, width=2, direction='in')
            ax0.tick_params(which='minor', length=5, width=2, direction='in')
            if (jrow+1 != nrows): ax0.set_xlabel(None)
            iplot+=1
    plt.subplots_adjust(wspace=0,hspace=0.05)        
    plt.tight_layout(w_pad=0,h_pad=0)
plt.show()

In [ ]:
nplots = 4 # up to nslice
plot_absorption_ratio(f_gpy_abs,f_lara_abs,Emin=5*u.GeV,Emax=200*u.GeV,nplots=nplots)
plot_absorption_ratio(f_gpy_abs,f_lara_abs,Emin=200*u.GeV,Emax=10000*u.GeV,nplots=nplots)